# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
hv.extension('bokeh') 

# Import API key
from api_keys import geoapify_key
params = {"units": "metric"}

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,27.03,74,75,5.14,CK,1733182209
1,1,jamestown,42.0970,-79.2353,-2.20,96,100,5.66,US,1733181951
2,2,albany,42.6001,-73.9662,-0.64,56,41,2.30,US,1733182141
3,3,guerrero negro,27.9769,-114.0611,19.09,62,57,4.64,MX,1733182216
4,4,grytviken,-54.2811,-36.5092,1.59,99,100,3.35,GS,1733182218


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]

)
    




# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)


]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

9


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,mendoza,-32.8908,-68.8272,24.74,19,0,2.57,AR,1733182021
30,30,tolanaro,-25.0319,46.9987,22.25,90,0,3.62,MG,1733182278
89,89,hwange,-18.3693,26.5019,22.57,37,0,0.90,ZW,1733182407
218,218,vallenar,-28.5708,-70.7581,21.66,35,0,2.15,CL,1733182691
256,256,harper,4.3750,-7.7169,26.01,82,0,2.61,LR,1733182779


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,mendoza,AR,-32.8908,-68.8272,19,
30,tolanaro,MG,-25.0319,46.9987,90,
89,hwange,ZW,-18.3693,26.5019,37,
218,vallenar,CL,-28.5708,-70.7581,35,
256,harper,LR,4.3750,-7.7169,82,
292,senanga,ZM,-16.1167,23.2667,23,
373,molina,CL,-35.1167,-71.2833,27,
536,inhambane,MZ,-23.8650,35.3833,75,
538,les avirons,RE,-21.2418,55.3394,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1


}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search
mendoza - nearest hotel: Hotel Eben-Ezer
tolanaro - nearest hotel: Hôtel Mahavokey
hwange - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
harper - nearest hotel: No hotel found
senanga - nearest hotel: Senanga Safari Lodge
molina - nearest hotel: La Cabaña
inhambane - nearest hotel: Hotel de Inhambane
les avirons - nearest hotel: Floralys


,City,Country,Lat,Lng,Humidity,Hotel Name
23,mendoza,AR,-32.8908,-68.8272,19,Hotel Eben-Ezer
30,tolanaro,MG,-25.0319,46.9987,90,Hôtel Mahavokey
218,vallenar,CL,-28.5708,-70.7581,35,Hotel del Marqués
292,senanga,ZM,-16.1167,23.2667,23,Senanga Safari Lodge
373,molina,CL,-35.1167,-71.2833,27,La Cabaña
536,inhambane,MZ,-23.8650,35.3833,75,Hotel de Inhambane
538,les avirons,RE,-21.2418,55.3394,64,Floralys


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Configure the map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Farhenheit Temperature",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 896,
    frame_height = 504,
    hover_cols = ["City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)